In [4]:
import bittensor
import torch
import torch.nn.functional as F
import torch.nn as nn
kl_loss = nn.KLDivLoss(reduction="batchmean")

# bittensor.logging( debug = True )
dataset = bittensor.dataset( dataset_name = ['Books3'] )
subtensor = bittensor.subtensor(  )
graph = bittensor.metagraph( subtensor = subtensor ).sync()
wallet = bittensor.wallet(name = 'default2', hotkey = 'default')
dendrite = bittensor.dendrite (wallet = wallet)
synapse = bittensor.TextCausalLM()
loss_fct = torch.nn.CrossEntropyLoss()

def cal_loss(inputs, query_response, validation_len = 1):
    _labels = inputs[:, 1:].contiguous()
    _logits = query_response[:, :-1, :].contiguous()
    loss = loss_fct(_logits.view(-1, _logits.size(-1)), _labels.view(-1))
    return loss

Output()

In [5]:
perm_uids = torch.randperm(graph.n)
random_endpoints = [graph.endpoints[uid] for uid in perm_uids[:100]]
inputs = next(dataset)
responses, return_ops, times = dendrite.text(
    endpoints=random_endpoints,
    inputs=inputs,
    synapses=[synapse],
    timeout=4
)

In [6]:
min([cal_loss(inputs, r[0]) for r in responses])

tensor(3.4665, grad_fn=<NllLossBackward0>)

In [12]:
losses = torch.tensor([cal_loss(inputs, r[0]).item() for r in responses])

In [22]:
losses_sorted = losses.sort()

print(list(zip(losses_sorted[1][:10], losses_sorted[0][:10])))
for i in range(2, 10, 2):
    mixed_responses = sum([responses[idx][0]/i for idx in losses_sorted[1][:i]])
    print(i, cal_loss(inputs, mixed_responses))


[(tensor(31), tensor(3.4665)), (tensor(98), tensor(3.4901)), (tensor(26), tensor(3.5527)), (tensor(25), tensor(3.5834)), (tensor(53), tensor(3.6258)), (tensor(8), tensor(3.6329)), (tensor(47), tensor(3.6936)), (tensor(79), tensor(3.6974)), (tensor(44), tensor(3.7064)), (tensor(84), tensor(3.7064))]
2 tensor(3.4315, grad_fn=<NllLossBackward0>)
4 tensor(3.3560, grad_fn=<NllLossBackward0>)
6 tensor(3.3608, grad_fn=<NllLossBackward0>)
8 tensor(3.3829, grad_fn=<NllLossBackward0>)


In [63]:
kl_loss = torch.nn.KLDivLoss(reduction="sum")
kl_losses = []
for idx in range(100):
    l = kl_loss(F.log_softmax(responses[31][0].view(-1, 50258)), F.softmax(responses[idx][0].view(-1, 50258)))
    # print(idx, l)
    kl_losses.append(l)

/tmp/ipykernel_1190605/411828831.py:4: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  l = kl_loss(F.log_softmax(responses[31][0].view(-1, 50258)), F.softmax(responses[idx][0].view(-1, 50258)))
/tmp/ipykernel_1190605/411828831.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  l = kl_loss(F.log_softmax(responses[31][0].view(-1, 50258)), F.softmax(responses[idx][0].view(-1, 50258)))


0 tensor(227.5915, grad_fn=<KlDivBackward0>)
1 tensor(965.2291, grad_fn=<KlDivBackward0>)
2 tensor(288.5288, grad_fn=<KlDivBackward0>)
3 tensor(965.2291, grad_fn=<KlDivBackward0>)
4 tensor(965.2291, grad_fn=<KlDivBackward0>)
5 tensor(227.5515, grad_fn=<KlDivBackward0>)
6 tensor(227.5915, grad_fn=<KlDivBackward0>)
7 tensor(227.5936, grad_fn=<KlDivBackward0>)
8 tensor(118.9006, grad_fn=<KlDivBackward0>)
9 tensor(229.4922, grad_fn=<KlDivBackward0>)
10 tensor(117.5225, grad_fn=<KlDivBackward0>)
11 tensor(227.5915, grad_fn=<KlDivBackward0>)
12 tensor(288.4880, grad_fn=<KlDivBackward0>)
13 tensor(965.2291, grad_fn=<KlDivBackward0>)
14 tensor(227.3867, grad_fn=<KlDivBackward0>)
15 tensor(131.3384, grad_fn=<KlDivBackward0>)
16 tensor(227.3867, grad_fn=<KlDivBackward0>)
17 tensor(227.3867, grad_fn=<KlDivBackward0>)
18 tensor(148.3501, grad_fn=<KlDivBackward0>)
19 tensor(121.3004, grad_fn=<KlDivBackward0>)
20 tensor(227.8950, grad_fn=<KlDivBackward0>)
21 tensor(131.3384, grad_fn=<KlDivBackward0>

In [59]:
import matplotlib.pyplot as plt

plt.plot(kl_losses, )

8 tensor(3.3770, grad_fn=<NllLossBackward0>)


In [27]:
# responses[31][0].shape
responses[31][0].view(-1, 50258)

torch.Size([200, 50258])

In [40]:
import torch.nn.functional as F
import torch.nn as nn
kl_loss = nn.KLDivLoss(reduction="batchmean")
# input should be a distribution in the log space
input = F.log_softmax(torch.randn(3, 5, requires_grad=True), dim=1)
# Sample a batch of distributions. Usually this would come from the dataset
target = F.softmax(torch.rand(3, 5), dim=1)
output = kl_loss(input, target)

print(input.shape, target.shape, output)

kl_loss = nn.KLDivLoss(reduction="batchmean", log_target=True)
log_target = F.log_softmax(torch.rand(3, 5), dim=1)
output = kl_loss(input, log_target)

print(input.shape, target.shape, output)

torch.Size([3, 5]) torch.Size([3, 5]) tensor(0.6772, grad_fn=<DivBackward0>)
torch.Size([3, 5]) torch.Size([3, 5]) tensor(0.6024, grad_fn=<DivBackward0>)


In [52]:
input

tensor([[0.0227, 0.1587, 0.3786, 0.0382, 0.4018],
        [0.0132, 0.1037, 0.1681, 0.0913, 0.6237],
        [0.0084, 0.1362, 0.0405, 0.6494, 0.1656]], grad_fn=<ExpBackward0>)